In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Spatio-temporal data processing for traffic safety/nlp

/content/drive/MyDrive/Spatio-temporal data processing for traffic safety/nlp


In [ ]:
!ls

sample_twitter_subway.csv		  subway_binary_model.pt
sentiment-traffic-model-classifier.ipynb  traffic-model-classifier.ipynb


In [ ]:
!pip install unidecode
!pip install tweet-preprocessor
!pip install contractions
!pip install word2number
!pip install transformers

     |████████████████████████████████| 245kB 8.4MB/s 
     |████████████████████████████████| 327kB 7.5MB/s 
     |████████████████████████████████| 286kB 13.0MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85396 sha256=ab301bd9c010d49fef73b249404f1f02dc81b03f884c36fee5f9f7ab54f8918c
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
  Created wheel for word2number: filename=word2number-1.1-cp37-none-any.whl size=5584 sha256=dc632f179a0be72f4f00d38ec963722de1d2ef16723c410aa271486e6cfa62c6
  Stored in directory: /root/.cache/pip/wheels/46/2f/53/5f5c1d275492f2fce1cdab9a9bb12d49286dead829a4078e0e
Successfully built word2number
     |████████████████████████████████| 2.5MB 7.6MB/s 
     |████████████████████████████████| 3.3MB 50.9MB/s 
     |████████████████████████████████| 901kB 48.0MB/s 


In [ ]:
import spacy
import pandas as pd
import unidecode
from word2number import w2n
import contractions
import preprocessor as p
import numpy as np


In [ ]:
sample_data_path = 'sample_twitter_subway.csv'
df_raw = pd.read_csv(sample_data_path)

# Preprocess

## Text Cleaning

Manual Elimination & Text Cleaning

Remove Retweets 'RT @' from the dataset, to lowercase and Remove all 'Uber Eats' records


In [ ]:
def remove_manually(df, word_list=['uber eats']):
    df = df[~df["txt"].str.contains("RT @")]
    df["txt"] = df["txt"].str.lower()
    for word in word_list:
        df = df[~df["txt"].str.contains(word)]
    return df

In [ ]:
df = remove_manually(df_raw)
df.replace({'label': {'n':0, 'y':1}}, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df

,Unnamed: 0,txt,label,sentiment
3,3,dancers often focus on training flexible hamst...,0,NaN
9,9,"@evanplewis after 14.5 months of silence, subw...",1,h
16,16,@zer0access @patriottakes trump one-upped jesu...,0,NaN
17,17,@ashindestad my mate used to say taking the tr...,1,h
19,19,"@laurenjohnston after 14.5 months of silence, ...",1,h
...,...,...,...,...
325,344,"do not go where the path may lead, go instead ...",0,NaN
326,345,jamaica–179 st/forest hills–71 av/145 st/norwo...,0,NaN
327,346,jamaica–179 st/forest hills–71 av/145 st/norwo...,0,NaN
328,348,@pingkind my expected path...the little top in...,0,NaN


Procedure
1.   Convert Accented Characters
2.   Expand Contractions
3.   Clean Text (using [tweet-preprocessor](https://github.com/s/preprocessor) package)
4.   Tokenize



In [ ]:
# https://towardsdatascience.com/nlp-text-preprocessing-a-practical-guide-and-template-d80874676e79
import en_core_web_sm
nlp = en_core_web_sm.load()

# exclude words from spacy stopwords list
deselect_stop_words = ['no', 'not']
for w in deselect_stop_words:
    nlp.vocab[w].is_stop = False

p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.HASHTAG, p.OPT.EMOJI, p.OPT.SMILEY)


def remove_accented_chars(text):
    """
    remove accented characters from text, e.g. café
    """
    text = unidecode.unidecode(text)
    return text

def expand_contractions(text):
    """
    expand shortened words, e.g. don't to do not
    """
    text = contractions.fix(text)
    return text

def clean_txt(target_text):

    target_text = remove_accented_chars(target_text)
    target_text = expand_contractions(target_text)
    target_text = p.clean(target_text)

    doc = nlp(target_text)
    clean_text = []
    for token in doc:
        flag = True
        edit = token.text
        # remove stop words
        if token.is_stop and token.pos_ != 'NUM': 
            flag = False
        # remove punctuations
        if  token.pos_ == 'PUNCT' and flag == True: 
            flag = False
        # remove special characters
        if  token.pos_ == 'SYM' and flag == True: 
            flag = False
        # remove numbers
        if (token.pos_ == 'NUM' or token.text.isnumeric()) and flag == True:
            flag = False
        # convert number words to numeric numbers
        if token.pos_ == 'NUM' and flag == True:
            edit = w2n.word_to_num(token.text)
        # convert tokens to base form
        elif token.lemma_ != "-PRON-" and flag == True:
            edit = token.lemma_
        # append tokens edited and not removed to list 
        if edit != "" and flag == True:
            clean_text.append(edit)
            
    return " ".join(clean_text)

## Word Embedding



In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
MAX_LEN = 110
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=clean_txt(sent),  # Preprocess sentence
            add_special_tokens = True,        # Add `[CLS]` and `[SEP]`
            max_length= MAX_LEN,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

# Training

In [ ]:
y = df.label
X = df.txt

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val =\
    train_test_split(X, y, test_size = 0.3, random_state=2021)

In [ ]:
import torch
cuda = torch.device('cuda')
device = torch.device("cuda")

In [ ]:
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert other data types to torch.Tensor
train_labels = torch.tensor(y_train.to_numpy())
val_labels = torch.tensor(y_val.to_numpy())

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 16

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, AdamW, get_linear_schedule_with_warmup

# Create the BertClassfier class
class BertClassifier(nn.Module):

    def __init__(self, freeze_bert=False):

        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H = 768, 50

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        #Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, 60),
            nn.Dropout(0.2),
            nn.Tanh(),
            nn.Linear(60, 1),
            nn.Sigmoid()
        )
        # self.classifier = nn.Sequential(
        #    nn.Linear(D_in, 1),
        #    nn.Sigmoid()
        # )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """

        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=1e-5,    # Default learning rate
                      eps=1e-5,    # Default epsilon value
                      weight_decay= 1)

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs
    print('total steps {}'.format(total_steps))

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [ ]:
import random
import time

# Specify loss function
criterion = nn.BCELoss().to(device)

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Train Acc':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*80)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            b_labels = b_labels.unsqueeze(1)
            # print(logits)
            # print(b_labels)
            loss = criterion(logits.double(), b_labels.double())
            # print(loss)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*80)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)
            train_acc = evaluate(model, train_dataloader)[1]
            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {train_acc:^9.2f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*80)
        print("\n")
    
    torch.save(model.state_dict(), '/content/drive/MyDrive/Spatio-temporal data processing for traffic safety/nlp/subway_binary_model.pt')
    

    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        b_labels = b_labels.unsqueeze(1)
        loss = criterion(logits.double(), b_labels.double())
        val_loss.append(loss.item())

        # Get the predictions

        preds = logits >= 0.5

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [ ]:
set_seed(12)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=20)
print('initialized')
train(bert_classifier, train_dataloader, val_dataloader, epochs=10, evaluation = True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


total steps 200
initialized
Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  | Train Acc  |  Val Acc  |  Elapsed 
--------------------------------------------------------------------------------
   1    |    9    |   0.661199   |     -      |     -     |     -     |   3.06   
--------------------------------------------------------------------------------
   1    |    -    |   0.661199   |  0.622520  |   71.67   |   76.25   |   4.54   
--------------------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  | Train Acc  |  Val Acc  |  Elapsed 
--------------------------------------------------------------------------------
   2    |    9    |   0.624942   |     -      |     -     |     -     |   3.00   
--------------------------------------------------------------------------------
   2    |    -    |   0.624942   |  0.593603  |   73.75   |   76.25   |   4.52   
----------------------------------------------------

KeyboardInterrupt: ignored

In [ ]:
pwd

'/content/drive/MyDrive/Spatio-temporal data processing for traffic safety/nlp'

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

In [ ]:
y.value_counts()